In [107]:
import geopandas as gpd
import pandas as pd
import vaex

In [108]:
df_iptu_distrito = vaex.open('data/IPTU-1995-2022-agrupados-por-distrito.hdf5')
df_iptu_subprefeitura = vaex.open('data/IPTU-1995-2022-agrupados-por-subprefeitura.hdf5')
df_iptu_od = vaex.open('data/IPTU-1995-2022-agrupados-por-od.hdf5')

In [109]:
gdf_distritos = gpd.read_file('data/SIRGAS_GPKG_distrito.gpkg')
gdf_distritos['area'] = gdf_distritos.area
gdf_distritos.geometry = gdf_distritos.simplify(tolerance=100)
gdf_distritos.to_crs(epsg=4674, inplace=True)

gdf_od = gpd.read_file('zip://data/SIRGAS_SHP_origemdestino_2017.zip!SIRGAS_SHP_origemdestino_2017', crs='EPSG:31983')
gdf_od = gdf_od[gdf_od.od_municip == '36']
gdf_od.set_crs(epsg=31983, inplace=True)
gdf_od['area'] = gdf_od.area
gdf_od.geometry = gdf_od.simplify(tolerance=10)
gdf_od.to_crs(epsg=4674, inplace=True)

gdf_subprefeitura = gpd.read_file('zip://data/SIRGAS_GPKG_subprefeitura.zip!SIRGAS_GPKG_subprefeitura.gpkg')
gdf_subprefeitura['area'] = gdf_subprefeitura.area
gdf_subprefeitura.geometry = gdf_subprefeitura.simplify(tolerance=100)
gdf_subprefeitura.to_crs(epsg=4674, inplace=True)

In [110]:
ano = [2002, 2022]
atributo = "Percentual de Uso Residencial"

In [5]:
gdf_agregacao = gdf_distritos.astype({'ds_codigo': 'int'})\
    .merge(df_iptu_distrito[df_iptu_distrito.ano >= ano[0] & df_iptu_distrito.ano <= ano[-1]].to_pandas_df(), \
        left_on='ds_codigo', right_on='distrito')\
            [["ds_codigo", "ds_nome", atributo, 'geometry', 'ano', 'Quantidade de Unidades']]

In [6]:
# (df_iptu_distrito.ano >= ano[0]) & (df_iptu_distrito.ano <= ano[1])

In [7]:
gdf_agregacao

,ds_codigo,ds_nome,Percentual de Uso Residencial,geometry,ano,Quantidade de Unidades
0,63,PIRITUBA,0.802808,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1995,30876
1,63,PIRITUBA,0.796576,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1996,31136
2,63,PIRITUBA,0.790158,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1997,31387
3,63,PIRITUBA,0.803951,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1998,32479
4,63,PIRITUBA,0.800241,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1999,32702
...,...,...,...,...,...,...
2683,37,ITAQUERA,0.835931,"POLYGON ((-46.43236 -23.52627, -46.42933 -23.5...",2018,43181
2684,37,ITAQUERA,0.835703,"POLYGON ((-46.43236 -23.52627, -46.42933 -23.5...",2019,43683
2685,37,ITAQUERA,0.835090,"POLYGON ((-46.43236 -23.52627, -46.42933 -23.5...",2020,44164
2686,37,ITAQUERA,0.816535,"POLYGON ((-46.43236 -23.52627, -46.42933 -23.5...",2021,45811


In [24]:
gdf_diff = gdf_agregacao.pivot(index='ds_codigo', columns='ano', values=atributo)

In [43]:
diff = (gdf_var[ano[-1]] - gdf_var[ano[0]])

In [48]:
gdf_distritos[atributo] = diff

In [49]:
gdf_distritos

,ds_codigo,ds_subpref,ds_cd_sub,ds_sigla,ds_nome,geometry,area,Percentual de Uso Residencial
0,63,PIRITUBA-JARAGUA,02,PIR,PIRITUBA,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1.712166e+07,NaN
1,95,PIRITUBA-JARAGUA,02,SDO,SAO DOMINGOS,"POLYGON ((-46.76387 -23.47407, -46.76027 -23.4...",9.886971e+06,0.062185
2,42,PIRITUBA-JARAGUA,02,JAR,JARAGUA,"POLYGON ((-46.77026 -23.45337, -46.76936 -23.4...",2.832976e+07,-0.005145
3,11,FREGUESIA-BRASILANDIA,03,BRL,BRASILANDIA,"POLYGON ((-46.71437 -23.46231, -46.71490 -23.4...",2.107930e+07,-0.173466
4,29,FREGUESIA-BRASILANDIA,03,FRE,FREGUESIA DO O,"POLYGON ((-46.68976 -23.50903, -46.70579 -23.5...",1.101703e+07,0.008886
...,...,...,...,...,...,...,...,...
91,44,SAO MIGUEL,23,JDH,JARDIM HELENA,"POLYGON ((-46.44027 -23.47986, -46.43500 -23.4...",9.143813e+06,0.027230
92,24,ITAQUERA,27,CLD,CIDADE LIDER,"POLYGON ((-46.51685 -23.55514, -46.51140 -23.5...",1.035511e+07,0.004018
93,57,ITAQUERA,27,PQC,PARQUE DO CARMO,"POLYGON ((-46.44461 -23.59272, -46.45600 -23.5...",1.545145e+07,0.041697
94,47,ITAQUERA,27,JBO,JOSE BONIFACIO,"POLYGON ((-46.42933 -23.53341, -46.42308 -23.5...",1.457084e+07,0.006954


In [111]:
ano = [2010, 2020]

In [134]:
gdf = gdf_od.astype({'od_id': 'int'})\
    .merge(df_iptu_od[(df_iptu_od.ano >= ano[0]) & (df_iptu_od.ano <= ano[-1])].to_pandas_df(), \
        left_on='od_id', right_on='od')\
            [["od_id", "od_nome", atributo, 'geometry', 'ano', 'Quantidade de Unidades']]
gdf_agregacao = gdf.loc[gdf.ano == ano[-1]]
diff = gdf.pivot(index='od_id', columns='ano', values=atributo)

gdf_diff = gdf_od.astype({'od_id': 'int'}).set_index('od_id').merge(diff, left_index=True, right_index=True, how='left')

# gdf_diff = gdf_od.set_index('od_id').copy()
gdf_diff.loc[:, atributo] = (gdf_diff[ano[-1]] - gdf_diff[ano[0]])
# gdf_diff.fillna(0.0, inplace=True)
gdf_agregacao.set_index('od_id', inplace=True)
# gdf_diff.set_index('od_id', inplace=True)
hover_data = ["od_nome"]
custom_data=["od_id"]
min_max = [df_iptu_od[atributo].min().item(), df_iptu_od[atributo].max().item()]
min_max_diff = [gdf_diff[atributo].min().item(), gdf_diff[atributo].max().item()]

In [135]:
gdf_diff

,od_nome,od_area,od_ano,od_municip,geometry,area,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Percentual de Uso Residencial
od_id,,,,,,,,,,,,,,,,,,
1,Sé,57.10,2017,36,"POLYGON ((-46.62926 -23.55037, -46.62774 -23.5...",5.696648e+05,0.056102,0.056165,0.056129,0.056098,0.056068,0.056659,0.064368,0.064311,0.064100,0.064043,0.062501,0.006399
2,Parque Dom Pedro,113.64,2017,36,"POLYGON ((-46.63537 -23.54303, -46.63417 -23.5...",1.133693e+06,0.123888,0.123260,0.123699,0.123326,0.124409,0.124389,0.161438,0.148013,0.148040,0.149436,0.124513,0.000625
3,Praça João Mendes,47.75,2017,36,"POLYGON ((-46.63311 -23.55653, -46.63481 -23.5...",4.763979e+05,0.327288,0.327287,0.328134,0.328822,0.329736,0.329791,0.341429,0.343347,0.350553,0.351410,0.357120,0.029832
4,Ladeira da Memória,75.11,2017,36,"POLYGON ((-46.63900 -23.54868, -46.63880 -23.5...",7.493191e+05,0.481503,0.485340,0.484359,0.487234,0.493691,0.492509,0.530302,0.538480,0.542386,0.550165,0.551287,0.069785
5,República,74.95,2017,36,"POLYGON ((-46.63658 -23.54449, -46.63639 -23.5...",7.477521e+05,0.253636,0.252411,0.252763,0.253347,0.253080,0.257814,0.263075,0.264641,0.266004,0.271001,0.285937,0.032301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,Jaguaré,430.44,2017,36,"POLYGON ((-46.74455 -23.55849, -46.74506 -23.5...",4.294572e+06,0.276735,0.276456,0.301603,0.303916,0.300822,0.380899,0.403425,0.409046,0.415022,0.415608,0.418375,0.141640
339,Cidade Universitária,584.56,2017,36,"POLYGON ((-46.74455 -23.55849, -46.74348 -23.5...",5.832153e+06,0.334091,0.332889,0.332057,0.328348,0.332556,0.331825,0.327242,0.326000,0.326267,0.325981,0.327179,-0.006912
340,Butantã,207.11,2017,36,"POLYGON ((-46.72322 -23.57260, -46.71533 -23.5...",2.066344e+06,0.649335,0.652004,0.647824,0.647917,0.608354,0.609289,0.607788,0.597123,0.597754,0.585185,0.581718,-0.067617


In [122]:
gdf_od.astype({'od_id': 'int'}).merge(gdf_diff).set_index('od_id')
# gdf_od.dtypes

,od_nome,od_area,od_ano,od_municip,geometry,area,Percentual de Uso Residencial
od_id,,,,,,,
1,Sé,57.10,2017,36,"POLYGON ((-46.62926 -23.55037, -46.62774 -23.5...",5.696648e+05,0.000000
2,Parque Dom Pedro,113.64,2017,36,"POLYGON ((-46.63537 -23.54303, -46.63417 -23.5...",1.133693e+06,0.006399
3,Praça João Mendes,47.75,2017,36,"POLYGON ((-46.63311 -23.55653, -46.63481 -23.5...",4.763979e+05,0.000625
4,Ladeira da Memória,75.11,2017,36,"POLYGON ((-46.63900 -23.54868, -46.63880 -23.5...",7.493191e+05,0.029832
5,República,74.95,2017,36,"POLYGON ((-46.63658 -23.54449, -46.63639 -23.5...",7.477521e+05,0.069785
...,...,...,...,...,...,...,...
338,Jaguaré,430.44,2017,36,"POLYGON ((-46.74455 -23.55849, -46.74506 -23.5...",4.294572e+06,0.000000
339,Cidade Universitária,584.56,2017,36,"POLYGON ((-46.74455 -23.55849, -46.74348 -23.5...",5.832153e+06,0.000000
340,Butantã,207.11,2017,36,"POLYGON ((-46.72322 -23.57260, -46.71533 -23.5...",2.066344e+06,0.000000


In [103]:
gdf_od

,od_nome,od_area,od_ano,od_municip,geometry,area
od_id,,,,,,
1,Sé,57.10,2017,36,"POLYGON ((-46.62926 -23.55037, -46.62774 -23.5...",5.696648e+05
2,Parque Dom Pedro,113.64,2017,36,"POLYGON ((-46.63537 -23.54303, -46.63417 -23.5...",1.133693e+06
3,Praça João Mendes,47.75,2017,36,"POLYGON ((-46.63311 -23.55653, -46.63481 -23.5...",4.763979e+05
4,Ladeira da Memória,75.11,2017,36,"POLYGON ((-46.63900 -23.54868, -46.63880 -23.5...",7.493191e+05
5,República,74.95,2017,36,"POLYGON ((-46.63658 -23.54449, -46.63639 -23.5...",7.477521e+05
...,...,...,...,...,...,...
338,Jaguaré,430.44,2017,36,"POLYGON ((-46.74455 -23.55849, -46.74506 -23.5...",4.294572e+06
339,Cidade Universitária,584.56,2017,36,"POLYGON ((-46.74455 -23.55849, -46.74348 -23.5...",5.832153e+06
340,Butantã,207.11,2017,36,"POLYGON ((-46.72322 -23.57260, -46.71533 -23.5...",2.066344e+06


In [99]:
gdf_od.loc[diff.index.to_list()]
# diff.index
# gdf_od

KeyError: "None of [Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,\n            ...\n            333, 334, 335, 336, 337, 338, 339, 340, 341, 342],\n           dtype='int64', name='od_id', length=337)] are in the [index]"

In [33]:
gdf.set_index('od_id', inplace=True)

,od_nome,Percentual de Uso Residencial,geometry,ano,Quantidade de Unidades
od_id,,,,,
1,Sé,0.056102,"POLYGON ((-46.62926 -23.55037, -46.62774 -23.5...",2010,7934
1,Sé,0.056165,"POLYGON ((-46.62926 -23.55037, -46.62774 -23.5...",2011,7938
1,Sé,0.056129,"POLYGON ((-46.62926 -23.55037, -46.62774 -23.5...",2012,8001
1,Sé,0.056098,"POLYGON ((-46.62926 -23.55037, -46.62774 -23.5...",2013,8006
1,Sé,0.056068,"POLYGON ((-46.62926 -23.55037, -46.62774 -23.5...",2014,8015
...,...,...,...,...,...
342,Jardim Bonfiglioli,0.545896,"POLYGON ((-46.73565 -23.58994, -46.73811 -23.5...",2014,16012
342,Jardim Bonfiglioli,0.545426,"POLYGON ((-46.73565 -23.58994, -46.73811 -23.5...",2015,16072
342,Jardim Bonfiglioli,0.556236,"POLYGON ((-46.73565 -23.58994, -46.73811 -23.5...",2016,16414


In [41]:
gdf = gdf_distritos.astype({'ds_codigo': 'int'})\
    .merge(df_iptu_distrito[(df_iptu_distrito.ano >= ano[0]) & (df_iptu_distrito.ano <= ano[-1])].to_pandas_df(), \
        left_on='ds_codigo', right_on='distrito')\
            [["ds_codigo", "ds_nome", atributo, 'geometry', 'ano', 'Quantidade de Unidades']]
# gdf.set_index('ds_codigo', inplace=True)
gdf_agregacao = gdf.loc[gdf.ano == ano[-1]]
diff = gdf.pivot(index='ds_codigo', columns='ano', values=atributo)
gdf_diff =  gdf_distritos.astype({'ds_codigo': 'int'}).copy()
gdf_diff[atributo] = (diff[ano[-1]] - diff[ano[0]])
gdf_diff.fillna(0.0, inplace=True)
hover_data = ["ds_nome"]
custom_data=["ds_codigo"]
min_max = [df_iptu_distrito[atributo].min().item(), df_iptu_distrito[atributo].max().item()]
min_max_diff = [gdf_diff[atributo].min().item(), gdf_diff[atributo].max().item()]

In [42]:
gdf_agregacao.set_index('ds_codigo', inplace=True)

In [43]:
gdf_agregacao

,ds_nome,Percentual de Uso Residencial,geometry,ano,Quantidade de Unidades
ds_codigo,,,,,
63,PIRITUBA,0.809992,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",2018,40592
95,SAO DOMINGOS,0.726268,"POLYGON ((-46.76387 -23.47407, -46.76027 -23.4...",2018,24850
42,JARAGUA,0.791802,"POLYGON ((-46.77026 -23.45337, -46.76936 -23.4...",2018,31503
11,BRASILANDIA,0.845535,"POLYGON ((-46.71437 -23.46231, -46.71490 -23.4...",2018,32018
29,FREGUESIA DO O,0.769560,"POLYGON ((-46.68976 -23.50903, -46.70579 -23.5...",2018,41784
...,...,...,...,...,...
44,JARDIM HELENA,0.861742,"POLYGON ((-46.44027 -23.47986, -46.43500 -23.4...",2018,20370
24,CIDADE LIDER,0.836785,"POLYGON ((-46.51685 -23.55514, -46.51140 -23.5...",2018,32620
57,PARQUE DO CARMO,0.707963,"POLYGON ((-46.44461 -23.59272, -46.45600 -23.5...",2018,14109
